# Open Source Software Development for Reliability and Lifetime Calculation
## TUMWVIB Chair of Vibroacoustics of Vehicles and Machines
#### Supervisors:  
* Sepahvand, Kheirollah; Dr.-Ing. habil.
* Kreuter, Daniel Christopher; Dr.-Ing
* Mueller, Johannes; Dr.-Ing

#### Student: 
Mustapha Kassem

# Bayes evaluation tool

## Initialization

In [ ]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy import stats, optimize
import mystic as my
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from os import path
import io
import sys, os
import json

sys.path.insert(0, os.path.abspath('..'))

from pylife.materialdata.woehler.fatigue_data import FatigueData
from pylife.materialdata.woehler.woehler_curve_creator import WoehlerCurveCreator


In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import warnings

## Bayes theory

\begin{equation}
P(H | D)=\frac{P(D | H) \, P(H)}{P(D)}
\end{equation}

- $P(H | D)$ is the posterior distribution, the sought probability of event $H$ occurring given that event $D$ is given or true.
- $P(D | H)$ is the likelihood distribution, the likelihood of event $D$ occurring given event $H$.
- $P(H)$ and $P(D)$ are the marginal probabilities of events $H$ and $D$ independent of one another.

- The posterior is equal to the likelihood of the data times the prior for the model parameters divided by a normalization constant. 
<br>
- If we have some domain knowledge, we can use it to assign priors for the model parameters, or we can use non-informative priors: distributions with large standard deviations that do not assume anything about the variable. 
<br>
- Using a non-informative prior means we “let the data speak.” A common prior choice is to use a normal distribution for β and a half-cauchy distribution for σ.

## To build a Bayesian model there are three steps

### 1. Define priors
The priors are the desired parameters that are to be sampled. They are defined as statistical distributions (normal, exponential etc). The statistical distribution is an assumption about the parameter

### 2. Define likelihood function(s)
The likelihood function is the function that defines the relationship the priors must fulfil in order to best fit the observed data of the model.

### 3. Inference
Lastly, the inference is where the number of samples are generated. The sampler used is called the No-U Turn Sampler (NUTS), which is a Markov chain Monte Carlo algorithm. The inference does not produce only one best fitted parameter, compared to the Maximum likelihood estimation, but a vast range of posterior distributions for the parameter.

In [ ]:
import pymc3 as pm
import theano
theano.gof.compilelock.set_lock_status(False)
import theano.tensor as tt

from pylife.materialdata.woehler.utils.bayes_analyzer import *

## WC model from FKM data base

In [ ]:
data = pd.read_excel("C:/Users/KSS7RNG/Desktop/Masterarbeit_KASSEM/Pylife/Test_dat.xlsx")
#data = pd.read_excel('../data/Test_dat.xlsx')
data.columns=['loads', 'cycles']

# Limit of the load cycle
ld_cyc_lim = data['cycles'].max()

# Class WoehlerCurve
fatigue_data = FatigueData(data, ld_cyc_lim)
woehler_curve_creator = WoehlerCurveCreator(fatigue_data)
Bayes_data = woehler_curve_creator.maximum_like_procedure({})


In [ ]:
x = np.log10(fatigue_data.fractures.loads)
y = np.log10(fatigue_data.fractures.cycles)

fig = plt.figure(figsize=(4,5.5))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model')
ax.plot(y, x, 'x', label='%d Failures'%(len(fatigue_data.fractures.loads)))
plt.plot(np.log10(fatigue_data.runouts.cycles), np.log10(fatigue_data.runouts.loads), 'bo', mfc='none', 
         label=u'%d Runouts'%(len(fatigue_data.runouts.loads)),alpha=0.5)
plt.xlabel("Load-cycle (N)", fontsize=11)
plt.ylabel("Load (S)", fontsize=11)
plt.legend(loc=0);
plt.ylim(2.44,2.58)
plt.xlim(min(y)*0.96,max(y)*1.04)
destination_path='C:\\Users\\KSS7RNG\\Desktop\\Masterarbeit_KASSEM\\Latex\\AMlatex\\AMStudentThesis\\results\\Bayes\\'
name = "bayes_fullsys_tight"
plt.savefig(destination_path+name+'.eps', format='eps', dpi=300)

## Sample data points from full system

In [ ]:
n_N=[0,0,0,0,0,0,0,5,0,5,0,5,0,5]
n_N_inf=[3,0,3,0,3,0,0]

ndraws = 3000  # number of draws from the distribution
nburn = 1000   # number of "burn-in points" (which we'll discard)

data_small_fin = ld_lvl_sample(Bayes_data, data, n_N)
data_small_inf, data_inf_dict, n_frac = ld_lvl_sample_inf(Bayes_data, n_N_inf)

data_total_x = np.concatenate((data_small_fin['x'], data_small_inf['x']))
data_total_y = np.concatenate((data_small_fin['y'], data_small_inf['y']))
data_total = {'x':10**data_total_x, 'y':10**data_total_y}

dataset = pd.DataFrame(data_total)
dataset.columns=['loads', 'cycles']

fatigue_data_sub = FatigueData(dataset)
woehler_curve_creator_sub = WoehlerCurveCreator(fatigue_data_sub)
Bayes_data_sub = woehler_curve_creator_sub.maximum_like_procedure({})


## Sub-system

In [ ]:
x_sub = np.log10(fatigue_data_sub.fractures.loads)
y_sub = np.log10(fatigue_data_sub.fractures.cycles)
data_sub = dict(x=x_sub, y=y_sub)

fig = plt.figure(figsize=(4, 5.5))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model')
ax.plot(y_sub, x_sub, 'x', label='%d Failures'%(len(fatigue_data_sub.fractures.loads)))
plt.plot(np.log10(fatigue_data_sub.runouts.cycles), np.log10(fatigue_data_sub.runouts.loads), 'bo', mfc='none', 
         label=u'%d Runouts'%(len(fatigue_data_sub.runouts.loads)), alpha=0.5)
plt.xlabel("Load-cycle (N)", fontsize=11)
plt.ylabel("Load (S)", fontsize=11)
plt.legend(loc=0)
plt.ylim(2.44,2.58)
plt.xlim(min(y)*0.96,max(y)*1.04)
destination_path='C:\\Users\\KSS7RNG\\Desktop\\Masterarbeit_KASSEM\\Latex\\AMlatex\\AMStudentThesis\\results\\Bayes\\'
name = "bayes_subsys_tight"
plt.savefig(destination_path+name+'.eps', format='eps', dpi=300)

# Bayesian tool models:

1. Slope $k_1$
2. Scatter in load-cycle direction $\frac{1}{T_N}$
3. Endurance $SD_{50}$ Scatter in load direction $\frac{1}{T_S}$

## Slope $k_1$

### 1. Define priors

In [ ]:
with pm.Model() as slope:
    sigma = pm.HalfCauchy('sigma', beta=10, testval=1.)
    intercept = pm.Normal('Intercept', 0, sigma=20)
    x_coeff = pm.Normal('x', 0, sigma=20)

### 2. Define likelihood

In [ ]:
with pm.Model() as slope:
    likelihood = pm.Normal('y', mu=intercept + x_coeff * x, sigma=sigma, observed=y_sub)

### 3. Inference!

In [ ]:
with pm.Model() as slope:
    trace_robust = pm.sample(3000,  chains=2)

### GLM built in function for linear regression in pymc3

In [ ]:
priors = {"sd": pm.HalfCauchy.dist(beta=10, testval=1.),
          "Intercept": pm.Normal.dist(mu=0, sigma=20),
          "x": pm.Normal.dist(mu=0, sigma=20)
          }
with pm.Model() as slope:
    pm.glm.GLM.from_formula('y ~ x', data_sub, priors = priors)
    trace_slope = pm.sample(6000, nuts_kwargs={'target_accept': 0.99}, chains=3, tune=1000)

In [ ]:
# Saving Fig
fig = pm.model_to_graphviz(slope)
destination_path='C:\\Users\\'
name = "slope_model"
fig.render(destination_path+name, view=False, format='eps')
fig

In [ ]:
burned_trace_slope = trace_slope[1000:]
pm.traceplot(burned_trace_slope)
# Saving Fig
fig = plt.gcf() # to get the current figure...
name = "k1_convergence"
fig.savefig(destination_path+name+'.eps', view=False, format='eps') 
plt.show()

In [ ]:
def trace_quantiles(x):

    return pd.DataFrame(pm.quantiles(x, [5, 50, 95]))

pm.stats.summary(burned_trace_slope, start=1000, stat_funcs=[trace_quantiles, trace_sd, monte]).round(2)

In [ ]:
bayesian_slope_plot(data_sub, burned_trace_slope, 
                    fatigue_data_sub.a_wl , fatigue_data_sub.b_wl, 
                    fatigue_data.a_wl, fatigue_data.b_wl)
name = "k1_samples_plot"
plt.savefig(destination_path+name+'.eps', format='eps', dpi=300)

In [ ]:
print('Full SYS %d data points:'%fatigue_data.fractures.shape[0])
print('Mali Value:         ', Bayes_data.curve_parameters['k_1'])
print('\nSub SYS %d data points:'% fatigue_data_sub.fractures.shape[0])
print('Bayes MC estimation:', abs(trace_quantiles(burned_trace_slope)[0]['x'][50]))
print('Mali Value:         ', Bayes_data_sub.curve_parameters['k_1'])

## Scatter $\frac{1}{T_N}$ model

In [ ]:
with pm.Model() as TN:
    
    # Priors
    stdev = pm.Exponential('stdev', lam=1/5)
    mu = pm.Normal('mu', mu=np.log10(fatigue_data_sub.N_shift).mean(), sd=10)
    
    # Likelihood
    y = pm.Normal('y', mu=mu, sd=stdev, observed=np.log10(fatigue_data_sub.N_shift))
    
    # Inference
    trace_TN = pm.sample(7000, nuts_kwargs={'target_accept': 0.99}, chains=4, tune=1000)

In [ ]:
# Saving fig
name = "TN_model"
fig_TN= pm.model_to_graphviz(TN)
fig_TN.render(destination_path+name, view=False, format='eps')
fig_TN

In [ ]:
burned_trace_TN = trace_TN[1000:]
pm.traceplot(burned_trace_TN)
# saving fig
fig = plt.gcf() # to get the current figure...
name = "TN_convergence"
fig.savefig(destination_path+name+'.eps', view=False, format='eps') 
plt.show()

In [ ]:
pm.stats.summary(trace_TN, start=1000, stat_funcs=[trace_quantiles, trace_sd, monte]).round(2)

In [ ]:
print('Full SYS %d data points:'%fatigue_data.fractures.shape[0])
print('Pearl chain:        ', fatigue_data.TN)
print('Mali Value:         ', Bayes_data.curve_parameters['1/TN'])
print('\nSub SYS %d data points:'% fatigue_data_sub.fractures.shape[0])
print('Bayes MC estimation:', trace_quantiles(burned_trace_TN)[0]['mu'][50])
print('Mali Value:         ', Bayes_data_sub.curve_parameters['1/TN'])
print('Pearl chain:        ', fatigue_data_sub.TN)

## Infinite Zone: $SD_{50}$, $TS_{50}$

In [ ]:
data_S = data_small_inf['x']
data_N = data_small_inf['y']

# create our Op
logl = LogLike(mali_sum_lolli, data_S, data_N, fatigue_data.load_cycle_limit)

# use PyMC3 to sampler from log-likelihood
with pm.Model() as inf_zone:
    
    # Priors
    SD = pm.Normal('SD_50', mu=data_S.mean(), sd=data_S.std()*5)
    TS = pm.Exponential('TS_50', lam=1/1.1)
    #TS = pm.Lognormal('TS_50', mu=np.log10(1.1), sd=np.log10(0.5))
    # convert SD and TS to a tensor vector
    var = tt.as_tensor_variable([SD, TS])

    # Likelihood
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': var})
    
    # Inference
    trace_SD_TS = pm.sample(5000, tune=1000, chains=3, discard_tuned_samples=True)

In [ ]:
pm.stats.summary(trace_SD_TS, stat_funcs=[trace_quantiles, trace_sd, monte]).round(2)

In [ ]:
print('Full SYS data points:',fatigue_data.zone_inf.shape[0])
print('Sub SYS data points:  ', len(data_small_inf['x']))

print('\nMali FUll Sys TS:', Bayes_data.curve_parameters['1/TS']) 
print('Bayes SUB Sys TS:',(trace_quantiles(trace_SD_TS)[0]['TS_50'][50]/2.5631031311))
print('Mali SUB Sys TS: ', Bayes_data_sub.curve_parameters['1/TS']) 

print('\nMali FUll Sys SD:', np.log10(Bayes_data.curve_parameters['SD_50']))
print('Bayes SUB Sys SD:',trace_quantiles(trace_SD_TS)[0]['SD_50'][50])
print('Mali SUB Sys SD: ', np.log10(Bayes_data_sub.curve_parameters['SD_50']))

# Results of bayes_multi_simulation Demo

In [ ]:
with open('bayes_dict.json', 'r') as fp:
    dict_import = json.load(fp)
dict_import

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of 1/TS')
ax.plot(np.arange(len(dict_import['Bayes_TS'])), 
        np.ones(len(dict_import['Mali_TS']))*np.mean(dict_import['Bayes_TS']),  'g')
ax.plot( np.arange(len(dict_import['Mali_TS'])),
        np.ones(len(dict_import['Mali_TS']))*np.mean(dict_import['Mali_TS']), 'r')

ax.plot(np.arange(len(dict_import['Bayes_TS'])),dict_import['Bayes_TS'],  'gx', label='Bayes_TS')
ax.plot( np.arange(len(dict_import['Mali_TS'])),dict_import['Mali_TS'], 'rx', label='Mali_TS')
ax.plot(np.arange(len(dict_import['Mali_TS'])),
        np.ones(len(dict_import['Mali_TS']))*Bayes_data.curve_parameters['1/TS'],'b', label='Full')
plt.legend(loc=0);

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of SD')
ax.plot(np.arange(len(dict_import['Bayes_SD'])), 
        np.ones(len(dict_import['Mali_SD']))*np.mean(dict_import['Bayes_SD']),  'g')
ax.plot( np.arange(len(dict_import['Mali_SD'])),
        np.ones(len(dict_import['Mali_SD']))*np.mean(dict_import['Mali_SD']), 'r')

ax.plot(np.arange(len(dict_import['Bayes_SD'])), dict_import['Bayes_SD'],  'gx', label='Bayes_SD')
ax.plot(np.arange(len(dict_import['Mali_SD'])),dict_import['Mali_SD'], 'rx', label='Mali_SD')
ax.plot(np.arange(len(dict_import['Mali_SD'])),
        np.ones(len(dict_import['Mali_SD']))*np.log10(Bayes_data.curve_parameters['SD_50']),'b', label='Full')
plt.legend(loc=0);

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of slope k')
ax.plot(np.arange(len(dict_import['Bayes_k'])), 
        dict_import['Bayes_k']*np.ones(len(dict_import['Bayes_k']))*-1,  'gx', label='Bayes_k')
ax.plot( np.arange(len(dict_import['Mali_k'])),dict_import['Mali_k'], 'rx', label='Mali_k')

ax.plot(np.arange(len(dict_import['Bayes_k'])), 
        np.mean(dict_import['Bayes_k'])*np.ones(len(dict_import['Bayes_k']))*-1,  'g')
ax.plot( np.arange(len(dict_import['Mali_k'])),
        np.mean(dict_import['Mali_k'])*np.ones(len(dict_import['Bayes_k'])), 'r')
ax.plot(np.arange(len(dict_import['Mali_k'])),
        np.ones(len(dict_import['Mali_k']))*Bayes_data.curve_parameters['k_1'],'b', label='Full')
plt.legend(loc=0);

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, xlabel='x', ylabel='y', title='Generated data and underlying model of 1/TN')
ax.plot(np.arange(len(dict_import['Bayes_TN'])), dict_import['Bayes_TN'],  'gx', label='Bayes_TN')
ax.plot( np.arange(len(dict_import['Mali_TN'])),dict_import['Mali_TN'], 'rx', label='Mali_TN')

ax.plot(np.arange(len(dict_import['Bayes_TN'])), 
        np.ones(len(dict_import['Mali_TN']))*np.mean(dict_import['Bayes_TN']),  'g')
ax.plot( np.arange(len(dict_import['Mali_TN'])),
        np.ones(len(dict_import['Mali_TN']))*np.mean(dict_import['Mali_TN']), 'r')
ax.plot(np.arange(len(dict_import['Mali_TN'])),
        np.ones(len(dict_import['Mali_TN']))*Bayes_data.curve_parameters['1/TN'],'b', 
        label='Full %f'%Bayes_data.curve_parameters['1/TN'])
#ax.set_yscale('log')
plt.legend(loc=0);